# TrendTracker: Exploring Trending Youtube Videos in Canada

We can now take the features we created and construct preliminary machine learning models. The target variable will be a feature we created, which was views per date until trending.

### Import Libraries and Data

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [51]:
# Change format
pd.options.display.float_format = '{:.5f}'.format

In [52]:
# File name represents most recent update
file = r'/Users/OliverPan/Desktop/youtube_data/trend_features.csv'

In [53]:
model_features = pd.read_csv(file)

/Users/oliverpan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (1,2,3,4,5,6,7,8,12,13,14,15,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [54]:
model_features.head()

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,...,comments_disabled,ratings_disabled,description,likes_to_dislikes,time_to_trend,youtube_link,no_description,views_per_date,num_trending,num_per_day
0,KX06ksuS6Xo,Diljit Dosanjh: CLASH (Official) Music Video |...,2020-08-11,UCZRdNleCgW-BGUJf-bbjzQg,Diljit Dosanjh,10.00000,2020-08-12,clash diljit dosanjh|diljit dosanjh|diljit dos...,9140911.00000,296541.00000,...,False,False,CLASH official music video performed by DILJIT...,47.98398,1.00000,https://www.youtube.com/watch?v=KX06ksuS6Xo,0.00000,9140911.00000,18.00000,230.00000
1,J78aPJ3VyNs,I left youtube for a month and THIS is what ha...,2020-08-11,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24.00000,2020-08-12,jacksepticeye|funny|funny meme|memes|jacksepti...,2038853.00000,353797.00000,...,False,False,I left youtube for a month and this is what ha...,134.62595,1.00000,https://www.youtube.com/watch?v=J78aPJ3VyNs,0.00000,2038853.00000,11.00000,230.00000
2,M9Pmf9AB4Mo,Apex Legends | Stories from the Outlands – “Th...,2020-08-11,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20.00000,2020-08-12,Apex Legends|Apex Legends characters|new Apex ...,2381688.00000,146740.00000,...,False,False,"While running her own modding shop, Ramya Pare...",52.51969,1.00000,https://www.youtube.com/watch?v=M9Pmf9AB4Mo,0.00000,2381688.00000,54.00000,230.00000
3,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22.00000,2020-08-12,brawadis|prank|basketball|skits|ghost|funny vi...,1514614.00000,156914.00000,...,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...,26.79085,1.00000,https://www.youtube.com/watch?v=3C66w5Z0ixs,0.00000,1514614.00000,12.00000,230.00000
4,VIUo6yapDbc,Ultimate DIY Home Movie Theater for The LaBran...,2020-08-11,UCDVPcEbVLQgLZX0Rt6jo34A,Mr. Kate,26.00000,2020-08-12,The LaBrant Family|DIY|Interior Design|Makeove...,1123889.00000,45803.00000,...,False,False,Transforming The LaBrant Family's empty white ...,47.51349,1.00000,https://www.youtube.com/watch?v=VIUo6yapDbc,0.00000,1123889.00000,11.00000,230.00000


In [55]:
model_features.columns

Index(['video_id', 'title', 'publishedAt', 'channelId', 'channelTitle',
       'categoryId', 'trending_date', 'tags', 'view_count', 'likes',
       'dislikes', 'comment_count', 'thumbnail_link', 'comments_disabled',
       'ratings_disabled', 'description', 'likes_to_dislikes', 'time_to_trend',
       'youtube_link', 'no_description', 'views_per_date', 'num_trending',
       'num_per_day'],
      dtype='object')

In [56]:
# Remove some variables temporarily
model_features = model_features[['views_per_date', 'categoryId', 'view_count', 'likes', 'dislikes', 'comment_count', 'comments_disabled', 'ratings_disabled','likes_to_dislikes', 'time_to_trend', 'no_description', 'num_trending']]

In [57]:
model_features.head()

,views_per_date,categoryId,view_count,likes,dislikes,comment_count,comments_disabled,ratings_disabled,likes_to_dislikes,time_to_trend,no_description,num_trending
0,9140911.00000,10.00000,9140911.00000,296541.00000,6180.00000,30059.00000,False,False,47.98398,1.00000,0.00000,18.00000
1,2038853.00000,24.00000,2038853.00000,353797.00000,2628.00000,40222.00000,False,False,134.62595,1.00000,0.00000,11.00000
2,2381688.00000,20.00000,2381688.00000,146740.00000,2794.00000,16549.00000,False,False,52.51969,1.00000,0.00000,54.00000
3,1514614.00000,22.00000,1514614.00000,156914.00000,5857.00000,35331.00000,False,False,26.79085,1.00000,0.00000,12.00000
4,1123889.00000,26.00000,1123889.00000,45803.00000,964.00000,2198.00000,False,False,47.51349,1.00000,0.00000,11.00000


In [58]:
# Change to categorical
model_features['comments_disabled'] = model_features['comments_disabled'].astype('category').cat.codes
model_features['ratings_disabled'] = model_features['ratings_disabled'].astype('category').cat.codes

In [59]:
model_features = model_features.replace([np.inf, -np.inf], np.nan)
model_features = model_features.dropna()

In [60]:
model_features.head()

,views_per_date,categoryId,view_count,likes,dislikes,comment_count,comments_disabled,ratings_disabled,likes_to_dislikes,time_to_trend,no_description,num_trending
0,9140911.00000,10.00000,9140911.00000,296541.00000,6180.00000,30059.00000,0,0,47.98398,1.00000,0.00000,18.00000
1,2038853.00000,24.00000,2038853.00000,353797.00000,2628.00000,40222.00000,0,0,134.62595,1.00000,0.00000,11.00000
2,2381688.00000,20.00000,2381688.00000,146740.00000,2794.00000,16549.00000,0,0,52.51969,1.00000,0.00000,54.00000
3,1514614.00000,22.00000,1514614.00000,156914.00000,5857.00000,35331.00000,0,0,26.79085,1.00000,0.00000,12.00000
4,1123889.00000,26.00000,1123889.00000,45803.00000,964.00000,2198.00000,0,0,47.51349,1.00000,0.00000,11.00000


In [61]:
model_features.corr()

,views_per_date,likes,dislikes,comment_count,comments_disabled,ratings_disabled,likes_to_dislikes,time_to_trend,no_description,num_trending
views_per_date,1.00000,0.77908,0.61631,0.67685,0.01930,nan,-0.03091,-0.07317,-0.01248,0.21446
likes,0.77908,1.00000,0.74878,0.80659,-0.01310,nan,0.03895,0.07161,-0.01212,0.19998
dislikes,0.61631,0.74878,1.00000,0.68032,0.01379,nan,-0.07745,0.06772,0.01970,0.10232
comment_count,0.67685,0.80659,0.68032,1.00000,-0.01350,nan,-0.01033,0.03108,-0.00482,0.10353
comments_disabled,0.01930,-0.01310,0.01379,-0.01350,1.00000,nan,-0.07513,0.04170,0.00775,-0.03837
ratings_disabled,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
likes_to_dislikes,-0.03091,0.03895,-0.07745,-0.01033,-0.07513,nan,1.00000,-0.07524,-0.01029,-0.06614
time_to_trend,-0.07317,0.07161,0.06772,0.03108,0.04170,nan,-0.07524,1.00000,0.10546,-0.01760
no_description,-0.01248,-0.01212,0.01970,-0.00482,0.00775,nan,-0.01029,0.10546,1.00000,-0.02232
num_trending,0.21446,0.19998,0.10232,0.10353,-0.03837,nan,-0.06614,-0.01760,-0.02232,1.00000


### Baseline Model: Linear Regression

We have a lot of colinearity between values so this model may be incorrect.

In [62]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [63]:
X_train, X_test, y_train, y_test = train_test_split(model_features[['views_per_date']], model_features.drop(['views_per_date'], axis=1), test_size=0.2, random_state=0)

In [64]:
model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [67]:
feature_importance = model.coef_

for i,j in enumerate(feature_importance):
    print('Feature: %0d, Score: %.5f' % (i,j))

Feature: 0, Score: -0.00000
Feature: 1, Score: 2.24594
Feature: 2, Score: 0.16563
Feature: 3, Score: 0.00566
Feature: 4, Score: 0.04410
Feature: 5, Score: 0.00000
Feature: 6, Score: 0.00000
Feature: 7, Score: -0.00000
Feature: 8, Score: -0.00000
Feature: 9, Score: -0.00000
Feature: 10, Score: 0.00000


In [ ]:
I be